In [24]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import yfinance as yf

from machine_learning_finance import (analyze_trades, 
    calc_probabilties_without_lookahead, plot_backtest_analysis, 
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for,
    calculate_polynomial_regression)

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "XLE"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()


def plot_polynomial(fig, df, name, chop=0):
    pred = calculate_polynomial_regression(df.head(len(df)-chop).copy())
    fig.add_trace(go.Scatter(x=df.index, y=pred, mode="lines", name=name))
    
def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df)  
    test_df = calc_probabilties_without_lookahead(test_df, hist_df)
    full_df = pd.concat([hist_df, test_df])
    
    
    fig = plot_backtest_analysis(full_df, ledger, inverse=symbol)
    
    chop = 5
    for i in range(60):
        plot_polynomial(fig, test_df, f"poly {chop}", chop)
        chop+=5
    
    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)

    fig.show()
    
    return ledger, test_df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[*********************100%***********************]  1 of 1 completed


In [25]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2022-01-10,61.761502,XLE,long,enter,NaN,4938.579999
1,2022-05-06,83.749198,XLE,long,exit,0.001113,6729.765889
2,2022-05-06,19.018300,DUG,long,enter,0.001113,6646.659875
3,2022-07-05,23.684501,DUG,long,exit,0.823721,8334.601812
4,2022-07-05,70.396997,XLE,long,enter,0.823721,8231.519311
5,2022-10-21,87.799300,XLE,long,exit,0.009385,10341.336314
6,2022-10-21,13.200700,DUG,long,enter,0.009385,10213.406276
7,2023-03-15,15.584300,DUG,long,exit,0.885892,12146.646708
8,2023-03-15,78.133601,XLE,long,enter,0.885892,11996.372091
9,2023-06-23,78.133601,XLE,long,exit,0.83559,12086.003321
